# 1. Imports e Configs

In [1]:
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Obtendo dados auxiliares

## 2.1 Informações dos locais de votação e dos Candidatos

In [2]:
########################################### Local de Votação ###########################################
# importando o excel com os dados 
df_loc_vot = pd.read_csv('\\eleitorado_local_votacao_2024.csv', encoding='latin1', sep = ';', dtype = {'CD_MUNICIPIO': 'string', 'NR_ZONA': 'string', 'NR_SECAO': 'string', 'NR_LOCAL_VOTACAO': 'string'})

# tratando os campos com as infos de zona e seção
df_loc_vot['NR_ZONA'] = df_loc_vot['NR_ZONA'].str.zfill(4)

df_loc_vot['NR_SECAO'] = df_loc_vot['NR_SECAO'].str.zfill(4)

########################################### Candidatos ###########################################
caminho_pasta = '\\consulta_cand_2024'

lista_dfs = []

for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.csv'):
        df = pd.read_csv(os.path.join(caminho_pasta, arquivo), encoding='latin1', sep = ';', dtype = {'NR_CANDIDATO': 'string', 'SG_UE': 'string', 'NR_PARTIDO': 'string'})
        lista_dfs.append(df)

df_cand = pd.concat(lista_dfs, ignore_index=True)

df_cand['SG_UF'] = df_cand['SG_UF'].str.lower()

### 2.1.2 Digite aqui o código do município, zona e seção que deseja.

In [3]:
print('Digite o código do município: ')
cod_municipio = input()

print('Digite a zona (4 Dígitos): ')
zona = input().zfill(4) 

print('Digite a seção (4 Dígitos): ')
secao = input().zfill(4)

uf = df_loc_vot[
    (df_loc_vot['CD_MUNICIPIO'] == cod_municipio) &
    (df_loc_vot['NR_ZONA'] == zona) &
    (df_loc_vot['NR_SECAO'] == secao)
].SG_UF.unique()[0].lower()

print('\n')
print(f'UF: {uf}')
print(f'Código do município: {cod_municipio}')
print(f'Zona: {zona}')
print(f'Seção: {secao}')

Digite o código do município: 


 13897


Digite a zona (4 Dígitos): 


 118


Digite a seção (4 Dígitos): 


 463




UF: ce
Código do município: 13897
Zona: 0118
Seção: 0463


# 3. Scraping

In [4]:
try:
    # configurando o navegador Chrome
    chrome_options = Options()
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_argument('--start-maximized')
    chrome_options.add_argument('--enable-javascript') # site do TSE usa javascript

    # WebDriver
    service = Service('\\chromedriver-win64\\chromedriver.exe')

    # inicializando o navegador com todas as preferências acima
    driver = webdriver.Chrome(service = service, options = chrome_options)

    # URL
    url = f'https://resultados.tse.jus.br/oficial/app/index.html#/eleicao;e=e619;uf={uf};mu={cod_municipio};ufbu={uf};mubu={cod_municipio};tipo=3;zn={zona};se={secao}/dados-de-urna/boletim-de-urna'
    driver.get(url)

    # ajuste técnico (rsrs) para renderizar o javascript
    boletim_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, 'Boletim de Urna'))
    )

    # clique no link de boletim de urna
    boletim_link.click()

    # redundancia para os dados terem carregados de fato
    identificacoes = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'mb-2') and contains(@class, 'box-identificacao')]"))
    )

    titulos = []
    valores = []

    # iterar sobre cada div encontrado acima
    for identificacao in identificacoes:
        
        titulo = identificacao.find_element(By.CLASS_NAME, 'titulo-sm').text

        valor = identificacao.find_element(By.CLASS_NAME, 'font-bold').text

        if titulo in ['Município', 'Zona Eleitoral', 'Seção Eleitoral', 'Local de votação']:
            titulos.append(titulo)
            valores.append(valor)

    dados_ident = pd.DataFrame({'Título': titulos, 'Valor': valores})

    dados_ident.set_index('Título', inplace = True)

    dados = []
        
    # localizando as linhas da tabela contendo as informacoes de voto por candidato
    linhas = driver.find_elements(By.XPATH, ".//following-sibling::div[contains(@class, 'row ng-star-inserted')]//div[contains(@class, 'flex justify-between v-line')]")

    # extração das linhas encotradas acima
    for linha in linhas:
        
        candidato = linha.find_element(By.XPATH, ".//p[contains(@class, 'mx-4+safe font-bold flex-1')]").text

        voto = linha.find_element(By.XPATH, ".//div[contains(@class, 'font-bold flex justify-end text-roxo')]").text

        dados.append({'NR_CANDIDATO': candidato, 'QTDE_VOTOS': voto})

    df = pd.DataFrame(dados)

    driver.quit()

    # adicionando os dados de identificacao para o df final
    df['COD_MUNICIPIO'] = dados_ident.loc['Município'].iloc[0]
    df['ZONA'] = dados_ident.loc['Zona Eleitoral'].iloc[0]
    df['SECAO'] = dados_ident.loc['Seção Eleitoral'].iloc[0]
    df['LOCAL_VOTACAO'] = dados_ident.loc['Local de votação'].iloc[0]

    print(f'Dados de Votação da Zona: {zona} e Seção: {secao} - OK!')

except Exception as e:
    # se ocorrer algum erro, encerrar o drive
    driver.quit()
    
    # exibe a mensagem de erro para a zona e seção
    print(f'Erro na Zona: {zona}, Seção: {secao}. Erro: {str(e)}')


Dados de Votação da Zona: 0118 e Seção: 0463 - OK!


## 3.1 Tratamento dos Resultados

In [5]:
# extraindo somente os valores númericos da info do candidato
df['NR_CANDIDATO'] = df['NR_CANDIDATO'].str.extract('(\d+)')

# extraindo informacoes dos candidatos
df = df.merge(df_cand[['NR_CANDIDATO', 'SG_UE', 'NM_CANDIDATO', 'NR_PARTIDO', 'NM_PARTIDO', 'NM_URNA_CANDIDATO', 'SG_UF', 'NM_UE']], left_on = ['NR_CANDIDATO', 'COD_MUNICIPIO'], right_on = ['NR_CANDIDATO', 'SG_UE'], how = 'left')

# transformacoes dos campos de uf e municipio
df['SG_UF'] = df['SG_UF'].str.upper()

df.rename(columns = {'SG_UF': 'UF', 'NM_UE': 'MUNICIPIO'}, inplace = True)

# inserindo infos dos locais de votação
df = df.merge(df_loc_vot[['CD_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NM_BAIRRO', 'NM_LOCAL_VOTACAO']], left_on = ['COD_MUNICIPIO', 'ZONA', 'SECAO'], right_on = ['CD_MUNICIPIO', 'NR_ZONA', 'NR_SECAO'], how = 'left')

# mantendo somente os campos necessarios
df = df[['NR_PARTIDO', 'NM_PARTIDO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'UF', 'MUNICIPIO', 'NM_BAIRRO', 'NM_LOCAL_VOTACAO', 'ZONA', 'SECAO', 'QTDE_VOTOS']]

# extraindo eventuais informacoes duplicados
df.drop_duplicates(inplace = True, ignore_index = True)

In [6]:
df.head()

,NR_PARTIDO,NM_PARTIDO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,UF,MUNICIPIO,NM_BAIRRO,NM_LOCAL_VOTACAO,ZONA,SECAO,QTDE_VOTOS
0,10,REPUBLICANOS,10001,XISTO SOARES DE OLIVEIRA,XISTO,CE,FORTALEZA,PASSARÉ,COLÉGIO LIRA COUTINHO,0118,0463,1
1,10,REPUBLICANOS,10121,NATANAEL CHARLES NUNES BRITO,CHARLES BARREIRA,CE,FORTALEZA,PASSARÉ,COLÉGIO LIRA COUTINHO,0118,0463,1
2,10,REPUBLICANOS,10123,RONALDO MANCHADO MARTINS,RONALDO MARTINS,CE,FORTALEZA,PASSARÉ,COLÉGIO LIRA COUTINHO,0118,0463,10
3,10,REPUBLICANOS,10190,EUVALDO REIS DA COSTA,EVALDO COSTA,CE,FORTALEZA,PASSARÉ,COLÉGIO LIRA COUTINHO,0118,0463,1
4,10,REPUBLICANOS,10224,JONAS ALVES CORDEIRO FILHO,JONAS FILHO,CE,FORTALEZA,PASSARÉ,COLÉGIO LIRA COUTINHO,0118,0463,1
